In [1]:
import tkinter as tk
from tkinter import scrolledtext, END
from datetime import datetime
from PIL import Image, ImageTk, ImageDraw
from RSA import *
from PlayFair import *
from database import *
from DES_Algorithm import *
import socket
import threading

class WhatsApp:
    def __init__(self, root):
        self.root = root
        self.root.title("WhatsApp")

        # Set green and white color scheme
        self.root.configure(bg="#25d366")  # Green background

        # Load and resize the profile picture using Pillow
        original_image = Image.open(r"C:\Users\small\Downloads\OIP.jpg")
        resized_image = original_image.resize((50, 50), Image.LANCZOS)  #Lanczos is a filter algorithm that is often used for high-quality image resizing

        # Create a circular mask for the image
        mask = Image.new('L', (50, 50), 0)
        draw = ImageDraw.Draw(mask)
        draw.ellipse((0, 0, 50, 50), fill=255)
        circular_image = Image.new('RGBA', (50, 50), (0, 0, 0, 0))
        circular_image.paste(resized_image, mask=mask)

        # Convert the circular image to PhotoImage
        self.profile_image = ImageTk.PhotoImage(circular_image)

        # Create a Label for the circular image
        self.image_label = tk.Label(root, image=self.profile_image, bg="#25d366")
        self.image_label.grid(row=0, column=0, padx=10, pady=0, sticky='w')  # Align to the left

        # Create a label for the name
        self.name_label = tk.Label(root, text="Hoss", bg="#25d366", fg="black", font=("Arial", 12, "bold"))
        self.name_label.grid(row=1, column=0, padx=10, pady=0, sticky='w')  # Align to the left
        
        # Create labels for video call and voice call symbols
        voice_call_icon = tk.Label(root, text="📞", bg="#25d366", fg="black", font=("Arial", 12))
        voice_call_icon.grid(row=1, column=1, padx=0, pady=0, sticky='e')  # Align to the right

        # Create a Listbox for displaying chat history
        self.chat_history = tk.Listbox(root, selectbackgroun="lightgreen", selectmode=tk.SINGLE, width=50, height=15, bg="white", fg="black", font=("Arial", 10))
        self.chat_history.grid(row=2, column=0, padx=10, pady=10, columnspan=2)
        
        # Create an entry widget for typing messages
        self.message_entry = tk.Entry(root, width=50, bg="white", fg="black", font=("Arial", 10))
        self.message_entry.grid(row=3, column=0, padx=10, pady=10, columnspan=2)

        # Create a button to send messages
        self.send_button = tk.Button(root, text="Send", command=self.send_message, bg="#25d366", fg="white", font=("Arial", 10, "bold"))
        self.send_button.grid(row=3, column=2, padx=10, pady=10)

    def send_message(self):
        
        # Get the message from the entry widget
        message = self.message_entry.get()
        
        encrypted_text = encrypt_des(message)
        
        user_data = {
            'user_id': 1,
            'message': encrypted_text
        }
        print('encrypted_in_main',encrypted_text)
        insert_user_data('chat_des', user_data)
        
        
        
        # Display the message in the chat history with a timestamp
        timestamp = datetime.now().strftime("%H:%M:%S")
        self.chat_history.insert(tk.END, f"You ({timestamp}): {message}\n")

        
        # Clear the entry widget
        self.message_entry.delete(0, END)

if __name__ == "__main__":
    # Create the main window
    root = tk.Tk()

    # connect to database
    connect_db()
    
    # Create an instance of the WhatsApp class
    chat_app = WhatsApp(root)
    
    # get messages history from database
    user_id_to_query = 1  # Replace with the desired user ID
    messages = get_user_messages_by_id(user_id_to_query, "chat_des")
    print(messages)
    for message in messages:
        # Assuming each sublist has 7 elements (adjust the number accordingly)
        element_count = 64
        original_list = [message[i:i+element_count] for i in range(0, len(message), element_count)]

        nested_list = [list(sublist) for sublist in original_list]

        print(nested_list)
        
        decrypted_message = decrypt_des(nested_list)
        chat_app.chat_history.insert(tk.END, f"You (time): {decrypted_message}\n") 
    # Run the Tkinter event loop
    root.mainloop()
    
    


Connected to MySQL database
Messages for ID 1: ['1000110000101010011001111110010101000110111001111010000111011110100101000101001000001100011101100110100100010100011001101100010111000111011110110110000010101001100001011100011100001000001111000101110011011001110010001101111101001111101011110111100000000010001010101011100111101100000010001111100111110011111000100101111110011111000010110101011110010001000110110100001000111110010000110110011101000101101010100011111111000100100000101101011111000000', '00011011011111010011000101011100010001000001001010100110011001011101010001110100111110000110101010000100000010101111110011000011', '1010111111101111110001000011000110111110011101110110010010011001000100011110100111011101010101010110000000111111111100001110110110110110110110111111100110011101011110010111101101101101001100100000101110011011111001111100101011111101001100110010111100110001011011000000010100011010010000100111110000101110000101001001010110011110010110101100100110000000011101100011011